## Short Term Wind Climate 
**5. Filter data: (report page 30 - 33)**
- Outliers in wind speed and direction 
- Suspicious data such as freezes 
- Datagaps 

**4. Interpolate:**
- Interpolate to the  one height of interest (= 150 meters for current turbine)

**7. Compare the both met masts:**
- Scatterplot: windspeed 150 m height mast one against windspeed 150m height mast 2 
- Same with the wind direction 
- Correlate with r^2 - linear correlation for both

**8. Short Term Wind Analysis**
- Mean, stdv of monthly and annual ws 
- Wind roses 
- Select parameters for Weibull model 
- Plot Weibull model 
- Diurnal profile of wind speed (blue) and wind direction (red) at HKW at 100 m MSL
- Monthly profile of the wind speed 
- Power curve of such a turbine
- Annual production of one turbine
- Annual production of wind farm


#### Import Libraries  

In [87]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import xarray as xr
import pandas as pd
import os
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np

#### File Paths

In [88]:
# Data Paths
bouy6_path = 'data/measurements/2023-11-06_Buoy6_BSH_N-9.nc'
bouy2_path = 'data/measurements/2023-11-09_Buoy2_BSH_N-9.nc'
era5_path = 'data/reanalysis/'
turbines_existing_path = 'data/turbine-info/coordinates/existing/'
turbines_planned_future_path = 'data/turbine-info/coordinates/planned_future/'
turbines_planned_in_operation_before_2030_path = 'data/turbine-info/coordinates/planned_in_operation_before_2030/'
turbines_planned_netherlands_path = 'data/turbine-info/coordinates/planned_netherlands/'
turbines_area_of_interest_path = 'data/turbine-info/coordinates/area_of_interest/'
shapefiles_DEU_path = 'data/shapefiles/DEU/*'
shapefiles_DNK_path = 'data/shapefiles/DNK/*'
shapefiles_NLD_path = 'data/shapefiles/NLD/*'
turbine_power_curves_path = 'data/turbine-info/power_curves/'

**3. Pre select measured data needed:**

**3.1 Wind Measurements:** 
- Wind speed
- wind direction 
- turbulence intensity

**3.2 Environmental Measurements:**
- Humidity 
- Pressure
- Temperature

**4. Pre Filter nc Files:**  
- select only the data needed for Enviroment and Wind Measurements
- select one height of interest (e.g. 94, or 140) for selected turbine Design

**5. set pre- filtered file in one dataframe**

From 'Overview' we decided to work with: buoy_6_measured and buoy_correlated_with_6 bcs. buoy 2 had a lot of data gaps (not a complete year measured)

In [89]:
#Load the datafiles in xr and netcdf 

xrbuoy6 = xr.open_dataset(bouy6_path)
xrbuoy2 = xr.open_dataset(bouy2_path)

buoy2_file = Dataset(bouy2_path)
buoy6_file = Dataset(bouy6_path)

In [90]:
# select the data of interest for now
time2 = xrbuoy2.variables['time'][:]
windspeed2 = buoy2_file.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:]
winddirection_buoy_2 = buoy2_file.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_from_direction'][:]

In [91]:
time6 = xrbuoy6.variables['time'][:]
windspeed6 = buoy6_file.groups['ZX_LIDAR_WLBZ_6'].variables['wind_speed'][:]
winddirection_buoy_6 = buoy6_file.groups['ZX_LIDAR_WLBZ_6'].variables['wind_from_direction'][:]

In [104]:
#Filter all NaN values consitent! 
def filter_nan_values(arr):
    return arr[~np.isnan(arr)]

def filter_time_nan(time_variable):
    valid_time_rows = time_variable.astype('str') != 'NaT'
    filtered_time_variable = time_variable[valid_time_rows]
    return filtered_time_variable

def filter_and_align_data(time_var, wind_speed_var, wind_direction_var):
    valid_indices = ~np.isnan(time_var) & ~np.isnan(wind_speed_var) & ~np.isnan(wind_direction_var)
    filtered_time = time_var[valid_indices]
    filtered_wind_speed = wind_speed_var[valid_indices]
    filtered_wind_direction = wind_direction_var[valid_indices]
    return filtered_time, filtered_wind_speed, filtered_wind_direction

def process_wind_data_nan(time2, winddirection_buoy_2, windspeed_buoy_2, time6, winddirection_buoy_6, windspeed_buoy_6):
    # Initialize empty lists to store the results
    wd_buoy_2_nanfilters = []
    ws_buoy_2_nanfilters = []
    wd_buoy_6_nanfilters = []
    ws_buoy_6_nanfilters = []
    time2_filtered = []
    time6_filtered = []

    # Loop over the range of indices and filter NaN values
    for i in range(0, 6):
        t2, ws2, wd2 = filter_and_align_data(time2, windspeed_buoy_2[:, 0, 0, i], winddirection_buoy_2[:, 0, 0, i])
        time2_filtered.append(t2)
        ws_buoy_2_nanfilters.append(ws2)
        wd_buoy_2_nanfilters.append(wd2)
        
        t6, ws6, wd6 = filter_and_align_data(time6, windspeed_buoy_6[:, 0, 0, i], winddirection_buoy_6[:, 0, 0, i])
        time6_filtered.append(t6)
        ws_buoy_6_nanfilters.append(ws6)
        wd_buoy_6_nanfilters.append(wd6)

    # Print the length of the original arrays
    print(f'The original length of time2 array: {len(time2)}')
    print(f'The original length of winddirection_buoy_2 array: {len(winddirection_buoy_2)}')
    print(f'The original length of windspeed_buoy_2 array: {len(windspeed_buoy_2)}')
    print(f'The original length of time6 array: {len(time6)}')
    print(f'The original length of winddirection_buoy_6 array: {len(winddirection_buoy_6)}')
    print(f'The original length of windspeed_buoy_6 array: {len(windspeed_buoy_6)}')

    # Print the length of the filtered arrays and absolute filtered values
    for i, (filtered_time, filtered_ws, filtered_wd) in enumerate(zip(time2_filtered, ws_buoy_2_nanfilters, wd_buoy_2_nanfilters)):
        print(f'The length of time2_filtered{i+1} array: {len(filtered_time)}')
        print(f'The length of ws_buoy_2_nanfilter{i+1} array: {len(filtered_ws)}')
        print(f'The length of wd_buoy_2_nanfilter{i+1} array: {len(filtered_wd)}')
        print(f'The absolute filtered wind direction values for wd_buoy_2_nanfilter{i+1}: {len(time2) - len(filtered_wd)}')

    for i, (filtered_time, filtered_ws, filtered_wd) in enumerate(zip(time6_filtered, ws_buoy_6_nanfilters, wd_buoy_6_nanfilters)):
        print(f'The length of time6_filtered{i+1} array: {len(filtered_time)}')
        print(f'The length of ws_buoy_6_nanfilter{i+1} array: {len(filtered_ws)}')
        print(f'The length of wd_buoy_6_nanfilter{i+1} array: {len(filtered_wd)}')
        print(f'The absolute filtered wind direction values for wd_buoy_6_nanfilter{i+1}: {len(time6) - len(filtered_wd)}')

    return time2_filtered, wd_buoy_2_nanfilters, ws_buoy_2_nanfilters, time6_filtered, wd_buoy_6_nanfilters, ws_buoy_6_nanfilters

# Example usage (assuming time2, winddirection_buoy_2, windspeed_buoy_2, time6, winddirection_buoy_6, and windspeed_buoy_6 are defined as numpy arrays)
time2_filtered, wd_buoy_2_nanfilters, ws_buoy_2_nanfilters, time6_filtered, wd_buoy_6_nanfilters, ws_buoy_6_nanfilters = process_wind_data_nan(time2, winddirection_buoy_2, windspeed2, time6, winddirection_buoy_6, windspeed6)


The original length of time2 array: 67680
The original length of winddirection_buoy_2 array: 67680
The original length of windspeed_buoy_2 array: 67680
The original length of time6 array: 57196
The original length of winddirection_buoy_6 array: 57196
The original length of windspeed_buoy_6 array: 57196
The length of time2_filtered1 array: 56633
The length of ws_buoy_2_nanfilter1 array: 56633
The length of wd_buoy_2_nanfilter1 array: 56633
The absolute filtered wind direction values for wd_buoy_2_nanfilter1: 11047
The length of time2_filtered2 array: 56901
The length of ws_buoy_2_nanfilter2 array: 56901
The length of wd_buoy_2_nanfilter2 array: 56901
The absolute filtered wind direction values for wd_buoy_2_nanfilter2: 10779
The length of time2_filtered3 array: 56047
The length of ws_buoy_2_nanfilter3 array: 56047
The length of wd_buoy_2_nanfilter3 array: 56047
The absolute filtered wind direction values for wd_buoy_2_nanfilter3: 11633
The length of time2_filtered4 array: 55494
The leng

In [105]:
#Example of usage 
print(wd_buoy_2_nanfilters[0][:200])

[102.639 103.927 104.599 107.635 108.704 107.512 109.771 110.212 110.578
 110.991 108.291 107.312 111.425 111.862 108.684 105.063 103.056 101.388
 101.722 100.489  99.469  97.175  99.615 100.122 102.824 101.823 100.011
  96.435 102.322  99.01  102.329 100.831 102.674 104.816 108.898 105.11
 105.507 108.115 107.609 107.102 106.649 109.965 110.739 104.334 108.727
 110.801 109.878 112.877 114.565 113.289 112.361 112.029 112.88  115.163
 115.109 114.667 119.073 122.424 120.836 117.306 118.024 117.896 117.17
 120.333 129.406 130.038 126.112 109.184 109.185 109.166 110.212 107.999
 110.5   108.103 113.481 111.584 109.294 112.23  108.179 105.681 107.449
 107.485 107.231 105.109 108.571 106.082 103.169 106.025 107.174 108.299
 102.897 101.473 103.912  96.731  95.275  99.513  98.667  98.506 103.848
 103.163 105.577 102.054 102.978 103.06  102.131  99.436 101.095 103.404
 104.809 100.308 105.893 101.167 102.745 104.822  99.187 105.237 106.13
 103.003 103.586 106.004 107.496 109.376 107.167 108.7